## Import
お決まりのimprot

In [ ]:
import sys
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

In [ ]:
import random

# CIFAR-10

## データセットのダウンロード

In [ ]:
train, test = datasets.get_cifar10(ndim=3)

データの中身を見てみる

In [ ]:
cifar10_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

sample = train[0]
image = sample[0]
label = sample[1]

plt.imshow(image.transpose((1, 2, 0)))
print('Label:', cifar10_labels[label])

## 【小課題】 DataAugmentationの実装
CIFAR-10のデータを対象に、

- Horizontal flipping
- Random cropping

を実装する

In [ ]:
def horizontal_flipping(img):
    ### Implement here!

    ###
    return flipped_img

In [ ]:
# Preview
img = train[0][0]
img = horizontal_flipping(img)
plt.imshow(img.transpose((1, 2, 0)))

In [ ]:
def random_cropping(img, crop_size):
    ### Implement here!

    ###
    return cropped_img

In [ ]:
# Preview
img = train[0][0]
img = random_cropping(img, 24)
plt.imshow(img.transpose((1, 2, 0)))

# CNNでImage classification

## モデル定義

例: LeNet5
https://docs.chainer.org/en/stable/tutorial/convnet.html#lenet5

In [ ]:
class LeNet5(Chain):
    def __init__(self, in_channels):
        super(LeNet5, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(
                in_channels=in_channels, out_channels=6, ksize=5, stride=1)
            self.conv2 = L.Convolution2D(
                in_channels=6, out_channels=16, ksize=5, stride=1)
            self.conv3 = L.Convolution2D(
                in_channels=16, out_channels=120, ksize=4, stride=1)
            self.fc4 = L.Linear(None, 84)
            self.fc5 = L.Linear(84, 10)

    def __call__(self, x):
        h = F.sigmoid(self.conv1(x))
        h = F.max_pooling_2d(h, 2, 1)
        h = F.sigmoid(self.conv2(h))
        h = F.max_pooling_2d(h, 2, 1)
        h = F.sigmoid(self.conv3(h))
        h = F.sigmoid(self.fc4(h))
        if chainer.config.train:
            return self.fc5(h)
        return F.softmax(self.fc5(h))

### 【課題】自分でCNNモデルを定義してみる
例：
```
Convolution (out_channels=32, k=5, s=1, p=0)
ReLU
BatchNormalization
MaxPooling (k=2, s=1)
Convolution (out_channels=64, k=3, s=1, p=0)
ReLU
BatchNormalization
Convolution (out_channels=64, k=3, s=1, p=0)
ReLU
BatchNormalization
MaxPooling (k=2, s=1)
FullyConnected (out_channels=512)
ReLU
FullyConnected (out_channels=10)
```

In [ ]:
class MyModel(Chain):
    ### Implement here!

    ###

### 【課題】 DataAugmentationを実装する

In [ ]:
class PreprocessedDataset(chainer.dataset.DatasetMixin):

    def __init__(self, base, mean=None, crop_size=30, random=True):
        if mean is None:
            mean = np.zeros((3, 1, 1))
        
        self.base = base
        self.mean = mean.astype('f').reshape((3, 1, 1))
        self.crop_size = crop_size
        self.random = random

    def __len__(self):
        return len(self.base)

    def get_example(self, i):
        image, label = self.base[i]

        ### Implement here!
        # If random == True, apply the following processings to a given image
        # 1. Random cropping
        # 2. Random horizontal flipping

        ###
        
        image -= self.mean # Subtract mean
        return image, label

In [ ]:
train = PreprocessedDataset(train)
test = PreprocessedDataset(test, random=False)

## 学習

In [ ]:
train_iter = iterators.SerialIterator(train, batch_size=100, shuffle=True)
test_iter = iterators.SerialIterator(test, batch_size=100, repeat=False, shuffle=False)

In [ ]:
model = L.Classifier(MyModel())

optimizer = optimizers.Adam()
optimizer.setup(model)

updater = training.StandardUpdater(train_iter, optimizer)
trainer = training.Trainer(updater, (20, 'epoch'), out='result')

trainer.extend(extensions.Evaluator(test_iter, model))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy', 'validation/main/accuracy']))
trainer.extend(extensions.ProgressBar())

trainer.run()

## 保存

In [ ]:
serializers.save_npz('my.model', model)